In [1]:
%pylab inline
from itertools import permutations, combinations_with_replacement

Populating the interactive namespace from numpy and matplotlib


In [87]:
def choose(n, k):
    if n < k:
        return 0
    
    else:
        num = math.factorial(n)
        den = math.factorial(k)*math.factorial(n - k)
    return int(num/den)

In [77]:
def gen_pi(K = 4, v = 20, acorn = 1234):
    np.random.seed(acorn)
    #Taken from Bijan
    
    while True:
        pi_0 = np.random.uniform(size=K) # Generate K Uniform(0,1) random variables
        pisum = np.sum(pi_0) # Calculate the sum of the K generated Uniform(0,1) random variables
        pi = [i/pisum for i in pi_0] # Normalize so that sum(pi) = 1
        Nv = [int(round(i*v)) for i in pi] # Round so that each block has an integer-valued amount of vertices
        if not 0 in Nv and np.sum(Nv)==v: # Make sure no block has 0 vertices and the sum of all vertices is correct
            break
    return pi, Nv # returns the vertex assignment distribution and the number of vertices in each block

In [17]:
def gen_Lambda(K, acorn=1234, ones_ = False):
    if ones_ == True:
        return np.ones(shape = (K, K))
    
    np.random.seed(acorn)
    Lambda = zeros(shape = (K, K)) # K x K matrix to store adjacency probabilities
    for i in range(K): # for each block
        for j in range(i, K): # for each combination (with replacement)
            Lambda[i, j] = np.random.uniform() # generate a Uniform(0,1) random variable
            Lambda[j, i] = Lambda[i, j] # Lambda is symmetric
                   
    return Lambda # returns a K x K, symmetric matrix where Lambda[i,j] in (0, 1)

In [18]:
def adj_matrix(n, pi, Lambda, acorn = 1234):
    np.random.seed(acorn)
    n = int(n) # Just in case!
    A = np.zeros(shape = (n, n)) # n x n adjcacency matrix
    K = len(pi) # extract the number of blocks in the SBM
    
    i = 0 # start at block indexed with 0
    while i < K: # while the block number is less than the total number of blocks
        for k in range(int(round(n*(sum(pi[:i])))), int(round(n*(sum(pi[:i + 1]))))): # for all vertices in block i
            c = i # start at block i
            while c < K: # while the block number is less than the total number of blocks
                for j in range(int(round(n*(sum(pi[:c])))), int(round(n*(sum(pi[:c + 1]))))): # for all vertices in block c
                    A[k, j] = np.random.binomial(1, Lambda[i, c]) # generates and assigns an edge based on block membership
                    A[j, k] = A[k, j] # A is symmetric
                c += 1
            A[k,k] = 0 # A is hollow
        i += 1
        
    return A # returns an n x n, symmetric and hollow matrix where A[i,j] in {0, 1}

In [19]:
def gen_seeds(Nv, seed_ratio, acorn = 1234):
    np.random.seed(acorn)
    
    K = len(Nv)
    
    num_seeds = [int(round(seed_ratio*Nv)) for i in range(K)]
    seeds = [[] for i in m]
    for i in range(K):
        for j in range(num_seeds[i]):
            index = np.random.randint(j)
            

In [98]:
K = 3
v = 9
labeled_ratio = 0.25
m = int(round(labeled_ratio*v))
n = int(v - m)

pi, Nv = gen_pi(K, v, acorn = 123)
mv = [int(round(labeled_ratio*Nv[i])) for i in range(K)]
nv = [int(round((1- labeled_ratio)*Nv[i])) for i in range(K)]
L = gen_Lambda(4, acorn = 3, ones_ = True)
               
A = adj_matrix(v, pi, L, acorn = 100)
print(A)
print(Nv)

for p in permutations(arange(m, n + m), n):
    V = [p[int(sum(nv[:i])):int(sum(nv[:i + 1]))] for i in range(K)]
    #for i in range(len(Nv)):
    #    print(Nv[:i], Nv[:i + 1])
    #    print(sum(Nv[:i]), sum(Nv[:i + 1]))
    #    print(p[int(sum(Nv[:i])):int(sum(Nv[:i + 1]))])
    #print(V)
    e = gen_e(A, V)
    c = gen_c(Nv, e)
    break
    
print(e)
print(c)


[[0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 0. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0.]]
[5, 2, 2]
[[6. 8. 4.]
 [0. 1. 2.]
 [0. 0. 0.]]
[[4. 2. 6.]
 [0. 0. 2.]
 [0. 0. 1.]]


In [85]:
p[5:9], len(p)

((5, 6, 7, 8), 9)

In [97]:
def gen_e(A, perm):
    K = len(perm)
    e = np.zeros(shape = (K, K)) # create an ((K choose 2) + K) x ((K choose 2) + K)
        # to store e[k,l]
        
    #e_test = np.zeros(shape = (choose(len(V), 2) + K, choose(len(V), 2) + K))
            
    for i in range(K):
        for j in range(i + 1, K):
            for v in perm[i]:
                for s in perm[j]:
                    e[i,j] += A[v, s]
    
    for i in range(K):
        for j in range(len(V[i])):
            for k in range(j + 1, len(V[i])):
                e[i, i] += A[j, k]
            
    
    return e #, e_test

In [ ]:


for p in permutations(arange(m, n + m), n):
    V = [p[int(sum(n_k[:i])):int(sum(n_k[:i + 1]))] for i in range(K)]
    
    

In [51]:
def gen_c(Nv, e):
    K = len(Nv) # number of blocks
    c = np.zeros(shape = (K, K)) # create an ((K choose 2) + K) x ((K choose 2) + K)
    
    for i in range(K): # for each block..
        for j in range(i, K): # for every other block..
            if i == j: # diagonal entries
                c[i,j] = choose(Nv[i], 2) - e[i,i] # Max possible edges minus realized edges
            else:
                c[i,j] = (Nv[i] * Nv[j]) - e[i,j] # Max possible edges minus realized edges
            
    return c

In [ ]:




#def likelihood_n(V, e, c, p):
    